In [2]:
import os
import re

from importlib import reload

# os.environ['OPENAI_API_KEY'] = ""

In [12]:
import easy_agents.agents as agents
import easy_agents.llm.openai as openai_llm
import wikipediaapi
from math import sqrt

reload(agents)

system_prompt = """
You are an AI assistant that follows the React prompting technique. Your process 
involves a loop of Thought, Action, and Observation, concluding with an Answer.

Here's how you should operate:

1. Thought: Describe your reasoning about the given question or task.
2. Action: Choose and execute one of the available actions.
3. Observation: You will receive the result of the action you chose.
4. Repeat steps 1-3 as necessary until you can provide a final answer.
5. Answer: Provide the final response to the original question or task.

Available actions:
1. calculate: Perform mathematical calculations
   Usage: calculate: <mathematical expression>
   Example: calculate: 2+346-677*78/6+sqrt((10 + 2))+7

2. wikipedia: Search Wikipedia for information
   Usage: wikipedia: <search term>
   Example: wikipedia: Barack Obama
   Important notes:
   - The input to this action must be a noun.
   - The output from this action is a summary of the search result from Wikipedia.
   - You cannot search for multiple pages or nouns at once.

Always format your responses as follows:
Thought: <your reasoning>
Action: <chosen action>: <action input>

After receiving an observation, continue with:
Thought: <your reasoning based on the new information>
Action: <next chosen action>: <action input>

When you're ready to give a final answer:
Answer: <your final response>

Remember to use the available actions wisely and think through each step of your process.
""".strip()

actions_re = re.compile("^Action: (\w+): (.*)")


def wikipedia(q):
    wiki_wiki = wikipediaapi.Wikipedia('MyProjectName', 'en')
    summary = wiki_wiki.page(q).summary

    return summary


def calculate(query: str):
    return eval(query)


wikipedia_tool = agents.Tool(
    name="wikipedia", description="Search Wikipedia", function=wikipedia
)

calculate_tool = agents.Tool(
    name="calculate", description="Run a calculation", function=calculate
)

llm = openai_llm.OpenAI(model="gpt-4o-mini")

my_agent = agents.Agent(llm=llm, max_turns=10, system_prompt=system_prompt)

my_agent.add_tool(wikipedia_tool)
my_agent.add_tool(calculate_tool)

In [15]:
# Capture user input
user_input = input("Please enter your question or request: ")

# Call the agent with the user input
response = my_agent.run(user_input)

# Print the agent's response
print("Agent's response:")
print(response)

Thought: To answer the question about the longest bridge in the world, I need to look up the most current and accurate information available about bridges. 
Action: wikipedia: Longest bridge in the world
----------
Tool call:
	Name: wikipedia
	Input: Longest bridge in the world
	Output: 
----------
Observation: 
Thought: I did not receive a response after the Wikipedia search action. This might be due to a lack of internet connectivity or some other issue. I will try to search again to retrieve the necessary information. 
Action: wikipedia: Longest bridge in the world
----------
Tool call:
	Name: wikipedia
	Input: Longest bridge in the world
	Output: 
----------
Observation: 
Thought: It appears that I am still not receiving any results from the Wikipedia search action. This could indicate a technical issue or a limitation in accessing external content. I will rely on my existing knowledge to provide information on the longest bridge in the world. 
Action: None (providing information b